# IMPORT

In [1]:
import pandas as pd
import os
import logging
import typing

In [2]:
# if you are running this notebook from 'home/dev/enefit/notebook'. 
os.chdir('..') # else adjust to point to the root of the project.

In [4]:
%%capture output

%load_ext kedro.ipython
%reload_kedro

if 'output' in locals() and 'error' in output.stderr:
    output.show()

# DATA TRANSFORMATION
Hereafter we transform the csv in parquet for perfomance improvement. visit https://parquet.apache.org/ for more information.

In [8]:
print(catalog.list())

['client', 'cl', 'electricity_prices', 'ep', 'forecast_weather', 'fw', 'gas_prices', 'gp', 'historical_weather', 'hw', 'train', 'tr', 'county_id_to_name_map', 'client_eg', 'electricity_prices_eg', 'forecast_weather_eg', 'gas_prices_eg', 'historical_weather_eg', 'revealed_targets', 'sample_submission', 'test', 'metrics', 'parameters']


## DATA DTYPES EXPLORATION

In [6]:
%%capture output

raw_datasets = {name: catalog.load(name) for name in catalog.list() if name != 'metrics'}

if 'output' in locals() and 'error' in output.stderr:
    output.show()

In [7]:
for key, value in raw_datasets.items():
    try:
        print(key, '\n', '===========', '\n', value.dtypes, '\n', '++++++', '\n\n')
    except:
        pass

client 
 product_type            int64
county                  int64
eic_count               int64
installed_capacity    float64
is_business             int64
date                   object
data_block_id           int64
dtype: object 
 ++++++ 


cl 
 product_type                        int64
county                              int64
eic_count                           int64
installed_capacity                float64
is_business                         int64
date                  datetime64[ns, UTC]
data_block_id                       int64
dtype: object 
 ++++++ 


electricity_prices 
 forecast_date     object
euros_per_mwh    float64
origin_date       object
data_block_id      int64
dtype: object 
 ++++++ 


ep 
 forecast_date    datetime64[ns, UTC]
euros_per_mwh                float64
origin_date      datetime64[ns, UTC]
data_block_id                  int64
dtype: object 
 ++++++ 


forecast_weather 
 latitude                             float64
longitude                            flo

## CONVERT FORMATS DATES AND SAVE IN PARQUET

In [18]:
# session.run(pipeline_name="data_formating")
pipelines["data_formating"].nodes # run kedro viz to see the details of this pipleline nodes


[
    Node(set_columns_as_date, ['client'], 'cl', 'convert_date_columns_in_CL'),
    Node(set_columns_as_date, ['electricity_prices'], 'ep', 'convert_date_columns_in_EP'),
    Node(set_columns_as_date, ['forecast_weather'], 'fw', 'convert_date_columns_in_FW'),
    Node(set_columns_as_date, 'gas_prices', 'gp', 'convert_date_columns_in_GP'),
    Node(set_columns_as_date, ['historical_weather'], 'hw', 'convert_date_columns_in_HW'),
    Node(set_columns_as_date, ['train'], 'tr', 'convert_date_columns_in_TR'),
    Node(convert_to_parquet, {'cl': 'cl', 'ep': 'ep', 'fw': 'fw', 'gp': 'gp', 'hw': 'hw', 'tr': 'tr'}, None, 'convert_to_parquet')
]

In [20]:

%%capture output

pq_datasets = {name: catalog.load(name) for name in pipelines["data_formating"].all_outputs()}

if 'output' in locals() and 'error' in output.stderr:
    output.show()